In [40]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [54]:
# import dataset and load in a dataframe
customer_file = r'data\customers_tm1_e.csv'
transaction_file = r'data\transactions_tm1_e.csv'
cust_df = pd.read_csv(customer_file)
trans_df = pd.read_csv(transaction_file)

In [42]:
cust_df.head()

,customer_id,dob,state,start_balance,creation_date
0,91,1993-07-01,California,10180.56,2007-01-31
1,92,1985-12-05,New York,4757.68,2007-01-31
2,93,1987-11-19,Minnesota,6796.72,2007-01-31
3,94,1981-03-23,Minnesota,9870.48,2007-01-31
4,95,1970-04-06,California,12500.72,2007-01-31


In [89]:
trans_df.head()

,date,account_id,customer_id,amount,transaction_date,deposit,withdrawal,account_total
0,2007-01-31,24137947,91,3034.26,2007-01-31,3034.26,0.00,-2260.92
1,2007-01-31,24137947,91,-5295.18,2007-01-16,0.00,-5295.18,-5295.18
2,2007-02-28,24137947,91,0.00,2007-02-28,0.00,0.00,-2260.92
3,2007-03-31,24137947,91,-0.00,2007-03-30,0.00,-0.00,-2260.92
4,2007-03-31,24137947,91,-0.00,2007-03-11,0.00,-0.00,-2260.92


In [49]:
cust_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116016 entries, 0 to 116015
Data columns (total 5 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   customer_id    116016 non-null  int64  
 1   dob            116016 non-null  object 
 2   state          116016 non-null  object 
 3   start_balance  116016 non-null  float64
 4   creation_date  116016 non-null  object 
dtypes: float64(1), int64(1), object(3)
memory usage: 4.4+ MB


In [101]:
trans_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4977972 entries, 0 to 4977971
Data columns (total 8 columns):
 #   Column            Dtype         
---  ------            -----         
 0   date              datetime64[ns]
 1   account_id        int64         
 2   customer_id       int64         
 3   amount            float64       
 4   transaction_date  datetime64[ns]
 5   deposit           float64       
 6   withdrawal        float64       
 7   account_total     float64       
dtypes: datetime64[ns](2), float64(4), int64(2)
memory usage: 303.8 MB


In [46]:
is_missing = trans_df.isna()

missing_per_column = is_missing.sum(axis = 0) 
print(missing_per_column)              

date                0
account_id          0
customer_id         0
amount              0
transaction_date    0
deposit             0
withdrawal          0
dtype: int64


In [57]:
# replacing empty start balances with 0, but this should be edited with transaction data if the information is there.
cust_df['start_balance'].fillna(0, inplace=True)

In [50]:
print(f"Customer columns:\n{list(cust_df.columns)}\n")
print(f"Transactions columns:\n{list(trans_df.columns)}")

Customer columns:
['customer_id', 'dob', 'state', 'start_balance', 'creation_date']

Transactions columns:
['date', 'account_id', 'customer_id', 'amount', 'transaction_date', 'deposit', 'withdrawal']


In [80]:
#Adding an account_total column, by grouping by customer_id and sorting by transaction_date
trans_df['account_total'] = trans_df.sort_values(['customer_id', 'transaction_date'], ascending=True).groupby(['customer_id', 'account_id'])['amount'].cumsum()

In [87]:
trans_df['date'] = pd.to_datetime(trans_df['date'])
trans_df['transaction_date'] = pd.to_datetime(trans_df['transaction_date'])

In [97]:
month_trans = trans_df.groupby(['customer_id', trans_df.transaction_date.dt.month])["amount"].sum()

In [98]:
print(month_trans)

customer_id  transaction_date
91           1                  -2260.92
             2                      0.00
             3                      0.00
92           1                      0.00
             2                   1164.90
                                  ...   
116102       5                   -350.95
116103       5                  -1003.22
116104       5                  -1102.10
116105       5                   2246.93
116106       5                     56.99
Name: amount, Length: 1059635, dtype: float64


In [ ]:
merge_